In [ ]:
import time
import requests
import pandas as pd
from  bs4 import BeautifulSoup

In [ ]:
start = time.time()

In [ ]:
links = pd.read_csv('Data/ml-small/links.csv',dtype={0:'str',1:'str',2:'str'})

imdbId and tmdbId are identifiers used by http://www.imdb.com and https://www.themoviedb.org respectively. So creating vairables containing URL without the identifiers.

In [ ]:
tmdb='https://www.themoviedb.org/movie/'
imdb='http://www.imdb.com/title/tt'

Combining the ID with the respective links to get the URLs.

In [ ]:
def link_builder(df,new_col,col,link):
    if link=='tmdb':
        df.loc[df[col].notna(),new_col]=df.loc[df[col].notna(),col].apply(lambda x: link+(x))
    else:
        df.loc[df[col].notna(),new_col]=df.loc[df[col].notna(),col].apply(lambda x: link+(x)+'/')

In [ ]:
link_builder(links,'imdb_url','imdbId',imdb)
link_builder(links,'tmdb_url','tmdbId',tmdb)

In [ ]:
links.isna().sum()

In [ ]:
links.head()

In [ ]:
links.drop(['imdbId','tmdbId'],axis=1)

Using requests and BeautifulSoup4 packages to scrape the plot summaries from the respective websites.

In [ ]:
valid_type=(type(BeautifulSoup(requests.get(links['imdb_url'][0]).content,'html.parser').find(class_='inline canwrap').find('span')))
def plot_scapper(url,url_col,valid_type=valid_type):
    if url_col=='tmdb_url':
        element = BeautifulSoup(requests.get(url).content,'html.parser').find(class_='overview')
        if type(element)!=valid_type:
            return None
        else:
            return element.get_text()
    else:
        element = BeautifulSoup(requests.get(url).content,'html.parser').find(class_='inline canwrap').find('span')
        if type(element)!=valid_type:
            return None
        else:
            return element.text

In [ ]:
def plot_builder(df,plot_col,url_col):
    df.loc[df[url_col].notna(),plot_col]=df.loc[df[url_col].notna(),url_col].apply(plot_scapper,args=(url_col,))

In [ ]:
plot_builder(links,'imdb_summary','imdb_url')

In [ ]:
plot_builder(links,'tmdb_overview','tmdb_url')

In [ ]:
links.head()

In [ ]:
links.to_csv('url_n_plots.csv',index=False)

In [ ]:
end=time.time()

In [ ]:
print('Run time : {} minutes').format((end-start)/60)